In [7]:
from delimitpy import parse_input
from delimitpy import process_empirical


# Parse the configuration file

We will use the ModelConfigParser class to parse the information from the configuration. A full description of the configuration file, along with an example, is available [here](https://delimitpy.readthedocs.io/en/latest/usage/parsinginput.html). This file provides delimitpy with the information it needs to conduct a species delimitation analysis. 

In [3]:
config_parser = parse_input.ModelConfigParser("../../examples/test1/config.txt")
config_values = config_parser.parse_config()

# Process empirical data

Next, we will process our empirical data. The format for the empirical data is described in detail [here](https://delimitpy.readthedocs.io/en/latest/usage/parsinginput.html). 

There are three major steps:
1) Read data from fasta files and convert to a numpy array.
2) Choose values for down-projecting the Site Frequency Spectrum.
3) Build SFS with down-projection.
4) Determine the average number of SNPs used to construct the SFS.


## Step 1: Read data into numpy array

In [9]:
data_processor = process_empirical.DataProcessor(config=config_values)
empirical_array = data_processor.fasta_to_numpy()
print(f"Our input data has {empirical_array.shape[0]} individuals and {empirical_array.shape[1]} SNPs.")

Our input data has 30 individuals and 1038 SNPs.


## Step 2: Choose values for down-projection

SFS cannot be generated from datasets that include missing data. To circumvent this, we use a downsampling approach such as that described in Satler and Carstens [(2017)](https://doi.org/10.1111/mec.14137). We must choose thresholds for each populations (i.e., the minumum number of individuals that must be sampled for a SNP to be used.) To help with this, we use the function find_downsampling from the class DataProcessor. This function generates a dictionary that holds the number of SNPs that meet each threshold.

We will use a folded SFS, meaning that we will build the SFS based on minor allele frequencies.

Since our data should be phased, and we will simulate diploid individuals, we will only consider multiples of 2.

In [12]:
# generate dictionary with the number of SNPs at different sampling thresholds
empirical_2d_sfs_sampling = data_processor.find_downsampling(empirical_array)

# print threshold, SNP combos for thresholds with at least 1030 snps
minspns = 1030
min_filtered = {key: value for key, value in empirical_2d_sfs_sampling.items() if value >= minspns}
print(min_filtered)


{(8, 6, 6): 1034, (8, 6, 4): 1037, (8, 6, 2): 1037, (8, 4, 6): 1034, (8, 4, 4): 1037, (8, 4, 2): 1037, (8, 2, 6): 1034, (8, 2, 4): 1037, (8, 2, 2): 1037, (6, 6, 6): 1035, (6, 6, 4): 1038, (6, 6, 2): 1038, (6, 4, 6): 1035, (6, 4, 4): 1038, (6, 4, 2): 1038, (6, 2, 6): 1035, (6, 2, 4): 1038, (6, 2, 2): 1038, (4, 6, 6): 1035, (4, 6, 4): 1038, (4, 6, 2): 1038, (4, 4, 6): 1035, (4, 4, 4): 1038, (4, 4, 2): 1038, (4, 2, 6): 1035, (4, 2, 4): 1038, (4, 2, 2): 1038, (2, 6, 6): 1035, (2, 6, 4): 1038, (2, 6, 2): 1038, (2, 4, 6): 1035, (2, 4, 4): 1038, (2, 4, 2): 1038, (2, 2, 6): 1035, (2, 2, 4): 1038, (2, 2, 2): 1038}


## Step 3: Build SFS with down-projection

Based on the results above, we need to choose a threshold for down-sampling. We want to strike a balance between the number of SNPs and the number of individuals we keep in our final SFS. By keeping eight individuals from population A, six individuals from population B, and six individuals from population C, we can use data from 1034 SNPs.

Now, we are ready to build our SFS. Note that our actual SFS will contain fewer than 1034 SNPs because some SNPs will become invariable when we downsample. Since we are downsampling, we will create ten replicate SFS.

We will build a set of joint SFS (jSFS) between each pairwise combination of populations. Additionally, we will build a multidimensional SFS (mSFS). When building the mSFS, we can optionally choose to use a binning approach [(Smith et al., 2018)](https://doi.org/10.1111/mec.14223). This creates a coarser SFS, which can be helpful if we have a lot of individuals and not that many SNPs.

The code below will also print the number of SNPs used on average to construct the mSFS, which will be helpful information when deciding how many SNPs to use to build the SFS for the simulated data.

In [14]:
downsampling_dictionary = {"A":8, "B":6, "C":6}

# build 10 replicates of the 2d SFS
empirical_2d_sfs = data_processor.numpy_to_2d_sfs(empirical_array, downsampling=downsampling_dictionary, replicates = 10)

# build 10 replicates of the mSFS
empirical_msfs, average_snps = data_processor.numpy_to_msfs(empirical_array, downsampling=downsampling_dictionary, replicates = 10)

We used an average of 333.4 to construct the mSFS.


# Build the models, and draw parameters from priors

Next, we will use functions from the ModelBuilder class to generate a baseline set of models (without parameters). Then, we use draw_parameters to draw parameterized models from the parameter space. This will return a lists of lists. Each list will correspond to a model. For each model, we have a list of parameterized versions of that model, with parameters drawn from the priors defined by the user in the configuration and species tree files.

A full description of the models generated by delimitpy is available [here](https://delimitpy.readthedocs.io/en/latest/usage/buildingmodels.html).

In [4]:
# build the models
model_builder = generate_models.ModelBuilder(config_values)
model_builder.build_models()

# parameterize the models
parameterized_models, labels  = model_builder.draw_parameters()

print((f"Number of models: {len(parameterized_models)}"))
print((f"Number of replicates: {len(parameterized_models[0])}"))

NameError: name 'generate_models' is not defined